In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import load_workbook


In [2]:
# Defining range of draft classes
start_year = 2018
end_year = 2019

writer = pd.ExcelWriter('nba_draft.xlsx', engine='xlsxwriter')

# Push data to .csv file
for i in range(start_year, end_year + 1):
    url = "https://www.basketball-reference.com/draft/NBA_{}.html".format(i)
    html = urlopen(url)
    soup = BeautifulSoup(html)
    rows = soup.findAll('tr', limit=2)
    headers = [th.getText() for th in rows[1].findAll('th')]
    headers = headers[1:]
    rows = soup.findAll('tr')[2:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
        for i in range(len(rows))]
    stats = pd.DataFrame(player_stats, columns = headers)
    data = stats.head(60)
    stats.to_excel(writer, sheet_name='draft_data_{}'.format(i), index = False)
writer.save()